In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('nba.db')
conn.execute("""ATTACH 'covid19.db' AS covid19""")
conn.execute("""ATTACH 'twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH 'imdb.db' AS imdb""")

## 56. 從 nba 資料庫查詢截至 2021-03-31 的得分王（生涯總得分 points 最高）、助攻王（生涯總助攻 assists 最高）、籃板王（生涯總籃板 totReb 最高）、抄截王（生涯總抄截 steals 最高）以及阻攻王（生涯總阻攻 blocks 最高）。

* 不同種類，第一個想到 UNION
* 也可以拿 personId 當作 WHERE 的條件
* 上一筆跟下一筆沒辦法直接抓出來 -> 就先個別做出來，再垂直合併

In [3]:
find_max_stats_total_from_nba =\
"""
-- SQL 查詢語法起點

SELECT players.firstName,
       players.lastName,
       'assists' AS category,
       career_summaries.assists
  FROM career_summaries
  JOIN players
    ON career_summaries.personId = players.personId
 WHERE assists = (SELECT MAX(assists)
                    FROM career_summaries)
UNION ALL
SELECT players.firstName,
       players.lastName,
       'steals' AS category,
       career_summaries.steals
  FROM career_summaries
  JOIN players
    ON career_summaries.personId = players.personId
 WHERE steals = (SELECT MAX(steals)
                    FROM career_summaries)
UNION ALL
SELECT players.firstName,
       players.lastName,
       'blocks' AS category,
       career_summaries.blocks
  FROM career_summaries
  JOIN players
    ON career_summaries.personId = players.personId
 WHERE blocks = (SELECT MAX(blocks)
                    FROM career_summaries)    
UNION ALL
SELECT players.firstName,
       players.lastName,
       'totReb' AS category,
       career_summaries.totReb
  FROM career_summaries
  JOIN players
    ON career_summaries.personId = players.personId
 WHERE totReb = (SELECT MAX(totReb)
                    FROM career_summaries) 
UNION ALL
SELECT players.firstName,
       players.lastName,
       'points' AS category,
       career_summaries.points
  FROM career_summaries
  JOIN players
    ON career_summaries.personId = players.personId
 WHERE points = (SELECT MAX(points)
                    FROM career_summaries)  
                    

-- SQL 查詢語法終點
"""

max_stats_total_from_nba = pd.read_sql(find_max_stats_total_from_nba, conn)
max_stats_total_from_nba

,firstName,lastName,category,assists
0,Chris,Paul,assists,10051
1,Chris,Paul,steals,2296
2,Dwight,Howard,blocks,2173
3,Dwight,Howard,totReb,14071
4,LeBron,James,points,35283


## 57. 從 nba 資料庫查詢截至 2021-03-31 各球隊陣中場均得分大於等於 20 分（ppg >= 20）的球員人數

In [4]:
find_number_of_top_scorers_by_teams_from_nba =\
"""
-- SQL 查詢語法起點

SELECT teams.fullName AS team_name,
       COUNT(players.teamId) AS number_of_players
  FROM career_summaries
  JOIN players
    ON career_summaries.personId = players.personId
  JOIN teams
    ON players.teamId = teams.teamId
 WHERE ppg >= 20
 GROUP BY players.teamId
UNION ALL 
SELECT teams.fullName as team_name,
       0 AS number_of_players
  FROM teams
 WHERE teams.fullName NOT IN (SELECT teams.fullName AS team_name
                                FROM career_summaries
                                JOIN players
                                  ON career_summaries.personId = players.personId
                                JOIN teams
                                  ON players.teamId = teams.teamId
                               WHERE ppg >= 20
                               GROUP BY players.teamId)
 ORDER BY number_of_players DESC;

-- SQL 查詢語法終點
"""

number_of_top_scorers_by_teams_from_nba = pd.read_sql(find_number_of_top_scorers_by_teams_from_nba, conn)
number_of_top_scorers_by_teams_from_nba

,team_name,number_of_players
0,Brooklyn Nets,4
1,Washington Wizards,2
2,Portland Trail Blazers,2
3,Los Angeles Lakers,2
4,Utah Jazz,1
5,San Antonio Spurs,1
6,Phoenix Suns,1
7,Philadelphia 76ers,1
8,Minnesota Timberwolves,1
9,Milwaukee Bucks,1


### 老師解答

* 使用 left join 保留需要的觀測值
* 使用 IFNULL 把空值轉成自己想要的值

In [5]:
find_number_of_top_scorers_by_teams_from_nba2 =\
"""
-- SQL 查詢語法起點

SELECT teams.fullName AS team_name,
       IFNULL(top_scorers_by_teams.number_of_players, 0) AS number_of_players  --空值的部分代換成想要給的值
  FROM teams
  LEFT JOIN (SELECT teams.fullName, -- LEFT JOIN : 以左表格的觀測值作為主體，沒加的話就是以「交集」為主
                    COUNT(*) AS number_of_players
                FROM career_summaries
                JOIN players
                  ON career_summaries.personId = players.personId
                JOIN teams
                  ON players.teamId = teams.teamId
               WHERE career_summaries.ppg >= 20
               GROUP BY teams.fullName) AS top_scorers_by_teams
         ON teams.fullName = top_scorers_by_teams.fullName
 ORDER BY number_of_players DESC,
          team_name 
-- SQL 查詢語法終點
"""

number_of_top_scorers_by_teams_from_nba2 = pd.read_sql(find_number_of_top_scorers_by_teams_from_nba2, conn)
number_of_top_scorers_by_teams_from_nba2

,team_name,number_of_players
0,Brooklyn Nets,4
1,Los Angeles Lakers,2
2,Portland Trail Blazers,2
3,Washington Wizards,2
4,Atlanta Hawks,1
5,Dallas Mavericks,1
6,Golden State Warriors,1
7,LA Clippers,1
8,Milwaukee Bucks,1
9,Minnesota Timberwolves,1


## 58. 從 twElection2020 資料庫查詢中國國民黨與民主進步黨在 2020 年選舉的得票率，包含總統副總統、不分區立委與區域立委，參考下列的預期查詢結果。¶

* 一步一步來，這題跟50題很像，就是先把各自的資料算出來之後，塞在FROM裡面當做要取的資料表，並且在SELECT的地方把他們取出來就好。
* 先把總統得票數的趴數算出來，在算其他的

In [6]:
summarize_kmt_dpp_votes_from_twelection2020 =\
"""
-- SQL 查詢語法起點

SELECT presidential_votes.party,
       presidential_votes.presidential,
       legislative_regional_votes.legislative_regional,
       legislative_at_large_votes.legislative_at_large
  FROM (SELECT parties.party,
               ROUND((CAST(SUM(votes) AS REAL) / (SELECT SUM(votes)
                                                    FROM presidential)) * 100, 2) || '%' AS presidential
          FROM presidential
          JOIN candidates
            ON candidates.id = presidential.candidate_id
          JOIN parties
            ON candidates.party_id = parties.id
         WHERE candidates.party_id IN (SELECT id
                                         FROM parties
                                        WHERE party IN ('中國國民黨', '民主進步黨')) AND
               candidates.type = '總統副總統'
         GROUP BY candidate_id) presidential_votes
  JOIN (SELECT parties.party,
               ROUND((CAST(SUM(legislative_regional.votes) AS REAL) / (SELECT SUM(votes) 
                                                                         FROM legislative_regional)) * 100, 2) || '%' AS legislative_regional 
          FROM legislative_regional
          JOIN candidates
            ON legislative_regional.candidate_id = candidates.id
          JOIN parties
            ON candidates.party_id = parties.id
         WHERE candidates.type = '區域立委' AND 
               candidates.party_id IN (SELECT id FROM parties WHERE party IN ('中國國民黨', '民主進步黨'))
         GROUP BY candidates.party_id) legislative_regional_votes
    ON presidential_votes.party = legislative_regional_votes.party
  JOIN (SELECT parties.party,
               ROUND((CAST(SUM(votes) AS REAL) / (SELECT SUM(votes) FROM legislative_at_large)) * 100, 2) || '%'  AS legislative_at_large
          FROM legislative_at_large
          JOIN parties
            ON legislative_at_large.party_id = parties.id
         WHERE legislative_at_large.party_id IN (SELECT id
                                                   FROM parties
                                                  WHERE party IN ('中國國民黨', '民主進步黨'))
         GROUP BY party_id)legislative_at_large_votes
    ON presidential_votes.party = legislative_at_large_votes.party;

-- SQL 查詢語法終點
"""

kmt_dpp_votes_from_twelection2020 = pd.read_sql(summarize_kmt_dpp_votes_from_twelection2020, conn)
kmt_dpp_votes_from_twelection2020

,party,presidential,legislative_regional,legislative_at_large
0,中國國民黨,38.61%,40.71%,33.36%
1,民主進步黨,57.13%,45.11%,33.98%


## 59. 從 twElection2020 資料庫查詢代表中國國民黨參選總統副總統的韓國瑜/張善政組合，在台灣 7,737 個選舉區（以村鄰里為一個選舉區）贏得的選舉區（得票數大於 > 蔡英文/賴清德組合）以及淨贏得票數

* 先把單一一個政黨的各個區域得票數算出來
* 把兩個政黨各個區域得票數 JOIN 在一起
* 在SELECT區塊將這兩個得票數相減即可

In [7]:
find_kmt_winning_villages_from_twelection2020 =\
"""
-- SQL 查詢語法起點

SELECT admin_regions.county,
       admin_regions.town,
       admin_regions.village,
       (kmt_votes - dpp_votes) AS net_winning_votes
  FROM(SELECT admin_region_id,
              SUM(votes) AS kmt_votes
         FROM presidential
        WHERE candidate_id = 2
       GROUP BY admin_region_id) AS kmt
  JOIN (SELECT admin_region_id,
               SUM(votes) AS dpp_votes
          FROM presidential
         WHERE candidate_id = 3
         GROUP BY admin_region_id) dpp
    ON kmt.admin_region_id = dpp.admin_region_id
  JOIN admin_regions
    ON kmt.admin_region_id = admin_regions.id
 WHERE net_winning_votes > 0
 ORDER BY net_winning_votes DESC;

-- SQL 查詢語法終點
"""

kmt_winning_villages_from_twelection2020 = pd.read_sql(find_kmt_winning_villages_from_twelection2020, conn)
kmt_winning_villages_from_twelection2020

,county,town,village,net_winning_votes
0,金門縣,金城鎮,西門里,2190
1,高雄市,左營區,海勝里,2096
2,臺北市,松山區,自強里,1802
3,桃園市,中壢區,自立里,1763
4,桃園市,龜山區,陸光里,1560
...,...,...,...,...
1327,基隆市,中山區,居仁里,1
1328,彰化縣,田中鎮,碧峰里,1
1329,新竹縣,湖口鄉,仁勢村,1
1330,澎湖縣,馬公市,東衛里,1
